In [1]:
%pip install SDV

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sdv

print(sdv.version.public)

1.11.0


In [3]:
from sdv.metadata import MultiTableMetadata

metadata = MultiTableMetadata()
metadata.detect_from_csvs(
    folder_name=('data')
)

metadata.update_column(
    table_name='Contractor',
    column_name ='ContractorId',
    sdtype='id'
)
metadata.update_column(
    table_name='Contractor',
    column_name ='SourceId',
    sdtype='id'
)

metadata.update_column(
    table_name='Competence',
    column_name ='CompetenceId',
    sdtype='id'
)
metadata.update_column(
    table_name='Competence',
    column_name ='SourceId',
    sdtype='id'
)
metadata.update_column(
    table_name='Contractor',
    column_name='BirthDay',
    sdtype='datetime',
    datetime_format='%Y%m%d'
)
metadata.update_column(
    table_name='Competence',
    column_name ='Acquired',
    sdtype='datetime',
    datetime_format='%Y-%m-%d %H:%M:%S.%f'
)
metadata.update_column(
    table_name='Competence',
    column_name ='Expires',
    sdtype='datetime',
    datetime_format='%Y-%m-%d %H:%M:%S.%f'
)
metadata.update_column(
    table_name='Competence',
    column_name ='Created',
    sdtype='datetime',
    datetime_format='%Y-%m-%d %H:%M:%S.%f'
)
metadata.update_column(
    table_name='Contractor',
    column_name ='ContractorUpdated',
    sdtype='datetime',
        datetime_format='%Y-%m-%d %H:%M:%S.%f'
)
metadata.update_column(
    table_name='Competence',
    column_name ='Contractor_Id',
    sdtype='id'
)
metadata.add_alternate_keys(
    table_name='Contractor',
    column_names=['ContractorId']
)
metadata.add_alternate_keys(
    table_name='Competence',
    column_names=['CompetenceId']
)
metadata.add_alternate_keys(
    table_name='Competence',
    column_names=['SourceId']
)
metadata.add_relationship(
    parent_table_name='Contractor',
    child_table_name='Competence',
    parent_primary_key='Id',
    child_foreign_key='Contractor_Id'
)

print("validate state: ", metadata.validate())
#metadata.visualize(
#    show_table_details='full',
#    show_relationship_labels=True,
#    output_filepath='my_metadata.png'
#)
metadata.save_to_json('TestdataJson1000')

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sdv/metadata/multi_table.py:518: UserWarning: Could not automatically add relationships for all tables. The relationships in the dataset are disjointed. Tables ['Competence', 'Contractor'] are not connected to any of the other tables.
  warnings.warn(warning_msg)


validate state:  None


In [4]:
from sdv.multi_table import HMASynthesizer
from sdv.datasets.local import load_csvs
from sdv.utils import drop_unknown_references

real_data = load_csvs( folder_name='data',
    read_csv_parameters={
        'skipinitialspace': True,
        'encoding': 'utf_8'
    })
#cleaned_data = drop_unknown_references(real_data, metadata)
synthesizer = HMASynthesizer(metadata)
synthesizer.fit(real_data)

synthetic_data = synthesizer.sample()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sdv/datasets/local.py:34: UserWarning: Ignoring incompatible files ['.amlignore', '.amlignore.amltmp'] in folder 'data'.
  warnings.warn(


InvalidDataError: The provided data does not match the metadata:
Table: 'Competence'
Error: Invalid values found for datetime column 'Expires': ['9999-12-31 23:59:59.9970000'].

In [ ]:
synthetic_data = synthesizer.sample(scale=0.1)

NYA

In [ ]:
import uuid

def format_columns_as_uuids(table, column_names):
    for column in column_names:
        table[column] = [str(uuid.uuid4()) for _ in range(len(table))]
    return table


synthetic_data['Contractor'] = format_columns_as_uuids(synthetic_data['Contractor'], ['ContractorId'])
synthetic_data['Competence'] = format_columns_as_uuids(synthetic_data['Competence'], ['CompetenceId', 'SourceId'])

synthetic_data['Competence']['CreatedBy'] = 'HMASynthesizer'
synthetic_data['Contractor']['PhotoUrl'] = 'HMASynthesizer'


from sdv.evaluation.multi_table import run_diagnostic

diagnostic = run_diagnostic(
    real_data=real_data,
    synthetic_data=synthetic_data,
    metadata=metadata
)


print(diagnostic)

In [ ]:
synthetic_data['Competence'].head(3)

In [ ]:
real_data['Competence'].head(3)

In [ ]:
synthetic_data['Contractor'].head(3)

In [ ]:
real_data['Contractor'].head(3)

In [27]:
from sdv.evaluation.multi_table import run_diagnostic

diagnostic = run_diagnostic(
    real_data=real_data,
    synthetic_data=synthetic_data,
    metadata=metadata
)

Generating report ...
(3/3) Evaluating Relationship Validity: : 100%|██████████| 1/1 [00:00<00:00, 113.05it/s]

Overall Score: 95.54%

Properties:
- Data Validity: 86.62%
- Data Structure: 100.0%
- Relationship Validity: 100.0%


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sdmetrics/single_column/statistical/boundary_adherence.py:58: RuntimeWarning: invalid value encountered in long_scalars
  return valid.sum() / len(synthetic_data)


In [28]:
from sdv.evaluation.multi_table import evaluate_quality

quality_report = evaluate_quality(
    real_data,
    synthetic_data,
    metadata
)

Generating report ...
(4/4) Evaluating Intertable Trends: : 100%|██████████| 130/130 [00:01<00:00, 114.63it/s]

Overall Score: 40.94%

Properties:
- Column Shapes: 50.26%
- Column Pair Trends: 46.83%
- Cardinality: 15.5%
- Intertable Trends: 51.19%


In [29]:
quality_report.get_details('Column Shapes')

,Table,Column,Metric,Score,Error
0,Competence,Source,TVComplement,1.000000e+00,None
1,Competence,Acquired,KSComplement,8.925671e-01,None
2,Competence,Lang,TVComplement,8.315054e-01,None
3,Competence,Qualification_Id,KSComplement,3.338070e-01,None
4,Competence,Created,KSComplement,1.899478e-02,None
5,Competence,CreatedBy,TVComplement,1.000000e-10,None
6,Competence,Modified,KSComplement,NaN,ValueError: Data passed to ks_2samp must not b...
7,Competence,ModifiedBy,KSComplement,NaN,ValueError: Data passed to ks_2samp must not b...
8,Contractor,Source,TVComplement,1.000000e+00,None
9,Contractor,PhotoUrl,TVComplement,1.000000e-10,None


In [ ]:
synthetic_data['Competence'].to_csv('competence_HMAdata_1000_100.csv', index=False)
synthetic_data['Contractor'].to_csv('contractor_HMAdata_1000_100.csv', index=False)